# PRIMMDebug Log Data Analaysis Notebook
This notebook displays all of the analysis of the log data that took place in the PRIMMDebug initial research paper.

The log data was collected from five schools between December 2024-February 2025. It is divided into the following sections:
1. **Summary statistics:** ...
2. **Establishing variables:**...
3. **Visualisation of variables:**...
4. **Students' written responses:**...

All you need to do is run the notebooks in order and the statistics that appear in the paper will be displayed. If there are any issues, please report them in the [Issues section of the GitHub repository](https://github.com/LaurieGale10/primmdebug-log-data-analysis/issues).

Before we run anything else, let's first import all of the necessary files.

In [8]:
from classes.ExerciseLog import ExerciseLog
from classes.StageLog import StageLog
from classes.StudentId import StudentId
from classes.processors.ExerciseLogProcessor import ExerciseLogProcessor
from classes.processors.StageLogProcessor import StageLogProcessor

from loading_services.fetch_log_from_firebase import *
from loading_services.fetch_logs_from_file import fetch_data_from_json

from constants import *

import plotly.express as px
import datetime

stage_logs: list[StageLog] = parse_stage_logs(fetch_data_from_json("data/stage_logs"))
exercise_logs: list[ExerciseLog] = parse_exercise_logs(stage_logs, fetch_data_from_json("data/exercise_logs")) #Cleaning should be done in parsing rather than here
student_ids: list[StudentId] = parse_student_ids(fetch_data_from_json("data/student_ids"))

## Summary Statistics

This data displays the following summary statistics to give information into the scale of the data we collected. We report below on:
- Number of exercises (that contain at least one completed PRIMMDebug stage)
  - Successful
  - Unsuccessful
  - Completed
  - Per each PRIMMDebug challenge
- Number of PRIMMDebug stages.
- Number of students
- Time of data collection


In [ ]:
print(f"Number of attempted PRIMMDebug challenges: {len(exercise_logs)}")

number_successful_exercises: int = 0
print(f"- Number of PRIMMDebug challenges where students reported successfully resolving the error they contained: {number_successful_exercises}")

number_unsuccessful_exercises: int = 0
print(f"- Number of PRIMMDebug challenges where students did not report successfully resolving the error they contained: {number_unsuccessful_exercises}")

#TODO: This line is totally unreadable; make this a function in the ExerciseLogProcessor class
number_completed_exercises: int = len([ExerciseLogProcessor.get_last_stage(exercise_log).stage_name for exercise_log in exercise_logs if ExerciseLogProcessor.get_last_stage(exercise_log) is not None and ExerciseLogProcessor.get_last_stage(exercise_log).stage_name == "modify"])
print(f"- Number of entirely completed PRIMMDebug challenges (where students reached the Make stage of PRIMMDebug): {number_completed_exercises}\n")

challenge_attempts: dict[str] = {}
for exercise_log in exercise_logs:
    if exercise_log.exercise_name not in challenge_attempts:
        challenge_attempts[exercise_log.exercise_name] = 1
    else:
        challenge_attempts[exercise_log.exercise_name] += 1
challenge_attempts_fig = px.bar(x = challenge_attempts.keys(), y = challenge_attempts.values(), labels = {"x": "Challenge Name", "y": "Frequency"})
challenge_attempts_fig.show()

from collections import Counter
challenge_end_stages: dict[str, int] = dict(Counter([ExerciseLogProcessor.get_last_stage(exercise_log).stage_name.name for exercise_log in exercise_logs if ExerciseLogProcessor.get_last_stage(exercise_log) is not None]))
final_stage_fig = px.bar(x = list(challenge_end_stages.keys()), y = list(challenge_end_stages.values()), labels = {"x": "Final stage of PRIMMDebug", "y": "Frequency"})
final_stage_fig.show()

print(f"Number of completed PRIMMDebug stages: {len(stage_logs)}")

print(f"Number of participating students: {len(student_ids)}")

gender_split_fig = px.bar(x = get_gender_split().keys(), y = get_gender_split().values(), labels = {"x": "Gender", "y": "Frequency"})
gender_split_fig.show()

year_group_split_fig = px.bar(x = get_year_group_split().keys(), y = get_year_group_split().values(), labels={"x": "Year Group", "y": "Frequency"})
year_group_split_fig.show()

school_split_fig = px.bar(x = get_school_split().keys(), y = get_school_split().values(), labels={"x": "School", "y": "Frequency"})
school_split_fig.show()

exercises_per_student: dict[str, int] = {}
for exercise in exercise_logs:
    student_id: str = exercise.student_id
    exercises_per_student[student_id] = exercises_per_student.get(student_id) + 1 if student_id in exercises_per_student else 1

attempted_challenges_per_student_fig = px.histogram(exercises_per_student.values(), marginal="box")
attempted_challenges_per_student_fig.show()


## Establishing Variables
Now we move onto introducing the variables that underpin our log data analysis. These include:
- Time taken
  - Per challenge attempt
  - Per stage
- Correctness of exercise
  - Per challenge
  - Per student
- Number of stages taken for a PRIMMDebug challenge
  - Per exercise
  - Per student

In [ ]:
print("Time taken (seconds)")
time_per_challenge_fig = px.histogram([ExerciseLogProcessor.get_time_on_exercise(exercise) for exercise in exercise_logs if hasattr(exercise,"end_time")], marginal="box", labels={"x": "Time taken (seconds)"})
time_per_challenge_fig.show()
time_per_stage_fig = px.histogram([StageLogProcessor.get_time_on_stage(stage) for stage in stage_logs if StageLogProcessor.get_time_on_stage(stage) is not None], marginal="box", labels={"x": "Time taken (seconds)"})
time_per_stage_fig.show()

print(" Correctness of PRIMMDebug challenges:")
print(f"- Per PRIMMDebug challenge")
print(f"- Per student")

print(" Number of stages taken on a PRIMMDebug challenge:")
stages_per_challenge_fig = px.histogram([len(exercise.stage_logs) for exercise in exercise_logs], marginal="box", labels={"x": "Number of stages"})
stages_per_challenge_fig.show()
#TODO: Segregate by each specific challenge (and student?)

## Exercise Log Stats
Placeholder for exercise log stats

In [ ]:
total_time: float = sum([ExerciseLogProcessor.get_time_on_exercise(exercise_log) for exercise_log in exercise_logs])
print(f"Total time on PRIMMDebug exercises: {datetime.timedelta(seconds=total_time)}")

final_program_states: list[bool] = [ExerciseLogProcessor.is_final_program_erroneous(exercise) for exercise in exercise_logs]
successful_final_program_states: list[bool] = [final_program_state for final_program_state in final_program_states if final_program_state]
proportion_successful_final_program_states: float = (len(successful_final_program_states) / len(final_program_states)) * 100
print(f"Proportion of PRIMMDebug challenges where last program run successfully executed: {proportion_successful_final_program_states:.2f}%")

print("Time spent focused on PRIMMDebug window per exercise")
time_spent_focused: list[float] = [ExerciseLogProcessor.get_time_focused(exercise) for exercise in exercise_logs]
time_spent_focused_fig = px.histogram(time_spent_focused, marginal="box", labels={"x": "100% Time spent focused on PRIMMDebug window"})
time_spent_focused_fig.show()

Total time on PRIMMDebug exercises: 1 day, 20:36:56.082000
Proportion of PRIMMDebug challenges where last program run successfully executed: 72.92%


## Written Responses

For now, just group written responses by stage name and investigate them. Also get some stats on written responses for context

In [ ]:
import enchant

from save_logs import *

save_written_responses(exercise_logs)

#Proprtion of reflections that contain do not contain at least one valid English word
dict = enchant.Dict("en_GB")
written_responses = ExerciseLogProcessor.get_written_response_data(exercise_logs)
print([response[3] for response in written_responses])

#Proportion of inspect the code stages containing written responses (should be modified to "including at least one valid word")